In [31]:
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
def upload_multiple_files():
    # Küçük tkinter penceresini gizle
    Tk().withdraw()

    # Birden fazla dosya seçmek için dosya seçici aç
    filenames = askopenfilenames(
        title="Dosyaları Seçin",
        filetypes=[("Tüm Dosyalar", "*.*")]  # İsteğe bağlı olarak dosya türlerini filtreleyebilirsiniz
    )

    return filenames



In [49]:
file_names=upload_multiple_files()

In [50]:
type(file_names[0])

str

## convert PDF content to text format

In [24]:
!pip install pypdf2 --quiet
from pypdf import PdfReader

In [51]:

def convert_PDF_Text(pdf_path):
  reader = PdfReader(pdf_path)
  pdf_texts = [p.extract_text().strip() for p in reader.pages]
  # Filter the empty strings
  pdf_texts = [text for text in pdf_texts if text]
  print("Document: ",pdf_path," chunk size: ", len(pdf_texts))
  return pdf_texts

In [52]:
pdf_texts = convert_PDF_Text(file_names[0])
pdf_texts[0]

Document:  C:/Users/ecoskun/Downloads/Bir Müslümanın Yol Haritası - Akademi A. Heyeti.pdf  chunk size:  839


'1. Giriş\n2. İman Esasları\n3. Kâinatı Yaratan Allah (cc)\n4. Nübüvve t (Peygamberlik Müessesesi)\n5. İnsanlığa Lütfedilen İlâhî Kitaplar\n6. Fizik Ötesi/Nuranî Varlıklar: Melekler\n7. İlâhî Takdir: Kader\n8. Ölüm Ötesi Sonsuz Hayat: Âhiret\n9. İbadet Mükellefiyeti\n10. Namaz\n11. Oruç\n12. Zekât\n13. Hac\n14. Helâl ve Haramlar\n15. Temizlik\n16. Ahlâk\n17. Kurban'

In [53]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [54]:

to_markdown(pdf_texts[0])

> 1. Giriş
> 2. İman Esasları
> 3. Kâinatı Yaratan Allah (cc)
> 4. Nübüvve t (Peygamberlik Müessesesi)
> 5. İnsanlığa Lütfedilen İlâhî Kitaplar
> 6. Fizik Ötesi/Nuranî Varlıklar: Melekler
> 7. İlâhî Takdir: Kader
> 8. Ölüm Ötesi Sonsuz Hayat: Âhiret
> 9. İbadet Mükellefiyeti
> 10. Namaz
> 11. Oruç
> 12. Zekât
> 13. Hac
> 14. Helâl ve Haramlar
> 15. Temizlik
> 16. Ahlâk
> 17. Kurban

In [55]:
print(file_names[0]," has ", len(pdf_texts), " pages")

C:/Users/ecoskun/Downloads/Bir Müslümanın Yol Haritası - Akademi A. Heyeti.pdf  has  839  pages


## convert Text from Pages to Chunks

In [56]:
!pip install langchain --quiet
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [58]:
def convert_Page_ChunkinChar(pdf_texts, chunk_size = 1500, chunk_overlap=0 ):
  character_splitter = RecursiveCharacterTextSplitter(
      separators=["\n\n", "\n", ". ", " ", ""],
      chunk_size=1500,
      chunk_overlap=0
)
  character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))
  print(f"\nTotal number of chunks (document splited by max char = 1500): \
        {len(character_split_texts)}")
  return character_split_texts

In [59]:
text_chunksinChar = convert_Page_ChunkinChar(pdf_texts)


Total number of chunks (document splited by max char = 1500):         1362


In [60]:

print("................. NOTICE ..................")
print(file_names[0]," has ", len(pdf_texts), " pages")
print(file_names[0]," has ", len(text_chunksinChar), " chunks")
print("chunk [0] has ", len(text_chunksinChar[0]), " chars")
print("chunk [-1] has ", len(text_chunksinChar[-1]), " chars")

................. NOTICE ..................
C:/Users/ecoskun/Downloads/Bir Müslümanın Yol Haritası - Akademi A. Heyeti.pdf  has  839  pages
C:/Users/ecoskun/Downloads/Bir Müslümanın Yol Haritası - Akademi A. Heyeti.pdf  has  1362  chunks
chunk [0] has  540  chars
chunk [-1] has  69  chars


convert Text from Chunks to Tokens

## Convert Text from Chunks to Tokens

This code is designed to tokenize the text chunks produced by the previous code using the `SentenceTransformersTokenTextSplitter`. Let's break down the code and its purpose:

1. `token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, model_name="distiluse-base-multilingual-cased-v1", tokens_per_chunk=128)`: This line initializes a tokenizer object named `token_splitter`. It utilizes the `SentenceTransformersTokenTextSplitter` class, which tokenizes text into chunks based on a specified model. The parameters provided are:
   - `chunk_overlap=0`: This parameter specifies that there should be no overlap between tokenized chunks.
   - `model_name="distiluse-base-multilingual-cased-v1"`: This parameter specifies the name of the pre-trained model to be used for tokenization. In this case, it's using the "distiluse-base-multilingual-cased-v1" model.

   - `tokens_per_chunk=128`: This parameter determines the maximum number of tokens allowed per chunk after tokenization.

2. `token_split_texts = []`: This line initializes an empty list to store the tokenized text chunks.

3. `for text in character_split_texts:`: This line starts a loop iterating over the text chunks produced by the character splitter.

4. `token_split_texts += token_splitter.split_text(text)`: Within the loop, this line tokenizes each text chunk using the `token_splitter` object and adds the resulting tokenized chunks to the `token_split_texts` list.

5. `print(token_split_texts[10])`: This line prints out the 11th tokenized chunk for inspection.

6. `print(f"\nTotal chunks: {len(token_split_texts)}")`: This line prints out the total number of tokenized chunks produced.

The reason for applying this code after the character splitting code is likely to further process the text data. Character splitting breaks down the text into smaller, more manageable chunks, and tokenization further breaks down those chunks into individual tokens, often for tasks like natural language processing (NLP) or machine learning. By applying both character splitting and tokenization, the text data becomes more structured and suitable for various NLP tasks.

In [71]:
!pip install sentence-transformers --quiet
!pip install tqdm --quiet

In [72]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
sentence_transformer_model="distiluse-base-multilingual-cased-v1"

In [73]:
def convert_Chunk_Token(text_chunksinChar,sentence_transformer_model, chunk_overlap=0,tokens_per_chunk=128 ):
  token_splitter = SentenceTransformersTokenTextSplitter(
      chunk_overlap=chunk_overlap,
      model_name=sentence_transformer_model,
      tokens_per_chunk=tokens_per_chunk)

  text_chunksinTokens = []
  for text in text_chunksinChar:
      text_chunksinTokens += token_splitter.split_text(text)
  print(f"\nTotal number of chunks (document splited by 128 tokens per chunk):\
       {len(text_chunksinTokens)}")
  return text_chunksinTokens

In [74]:
from tqdm import tqdm, trange


In [76]:
text_chunksinTokens = convert_Chunk_Token(text_chunksinChar,sentence_transformer_model)


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\ecoskun\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.